<a href="https://colab.research.google.com/github/BasselWA/NLPAshrafBasil/blob/main/m3Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q sumeval==0.2.2
!pip install -q nlpaug==1.1.3
!pip install -q simpletransformers==0.60.9


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip install nltk


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import libraries <a class="anchor" id="Import-libraries"></a>

In [ ]:
import gc
import random
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import nlpaug.augmenter.word as naw
from sumeval.metrics.rouge import RougeCalculator

import torch
from simpletransformers.t5 import T5Model, T5Args

print('Pytorch version: %s'  % torch.__version__)

d:\Semester 10\NLP\project\newnlproj\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Pytorch version: 2.3.0+cpu


In [ ]:
import pandas as pd

# Replace 'your_file.csv' with the actual name of your CSV file
file_path = "email_thread_details.csv"

# Read the CSV file into a DataFrame
email_threads_details = pd.read_csv(file_path)

# Replace 'your_file.csv' with the actual name of your CSV file
file_path = "email_thread_summaries.csv"

# Read the CSV file into a DataFrame
email_threads_summaries = pd.read_csv(file_path)
email_threads_details['timestamp'] = pd.to_datetime(email_threads_details['timestamp'])
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Merge the two datasets on 'thread_id'
email_data = pd.merge(email_threads_details, email_threads_summaries, on='thread_id', how='inner')

# Group emails by thread_id and concatenate the bodies
email_data_grouped = email_data.groupby('thread_id').agg({'body': ' '.join, 'summary': 'first'}).reset_index()

# Preprocess the data
# Remove any rows with missing values
email_data_grouped.dropna(inplace=True)

# Convert 'body' and 'summary' columns to lowercase
email_data_grouped['body'] = email_data_grouped['body'].str.lower()
email_data_grouped['summary'] = email_data_grouped['summary'].str.lower()


In [ ]:
df = email_data_grouped.dropna().reset_index(drop=True)

## EDA <a class="anchor" id="EDA"></a>

In [ ]:
display(df.head(1))

,thread_id,body,summary
0,1,\n\n -----original message-----\nfrom: =09ther...,the email thread discusses the master terminat...


In [ ]:
df = df.drop(['thread_id'], axis=1)

In [ ]:
df = df.rename(columns={'body': 'input_text', 'summary': 'target_text'}).reindex(columns=['input_text', 'target_text'])
df['prefix'] = ''

train, test = train_test_split(df, test_size=0.2, random_state=42)
train, valid = train_test_split(train, test_size=0.2, random_state=42)

## Build the model <a class="anchor" id="Build-the-model"></a>

### Training

In [ ]:
cuda =  torch.cuda.is_available()

In [ ]:
train_params = {
    'max_seq_length': 512,
    'max_length': 128,
    'train_batch_size': 8,
    'eval_batch_size': 8,
    'num_train_epochs': 2,
    'evaluate_during_training': True,
    'evaluate_during_training_steps': 10000,
    'use_multiprocessing': False,
    'fp16': False,
    'save_steps': -1,
    'save_eval_checkpoints': False,
    'save_model_every_epoch': False,
    'no_cache': True,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'preprocess_inputs': False,
    'num_return_sequences': 1
}

model = T5Model('t5', 't5-small', args=train_params, use_cuda=cuda)
model.train_model(train, eval_data=valid)
gc.collect()

d:\Semester 10\NLP\project\newnlproj\nlp\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
d:\Semester 10\NLP\project\newnlproj\nlp\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Omen\.cache\huggingface\hub\models--t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an admini

Using Adafactor for T5


Epochs 1/2. Running Loss:    2.3755: 100%|██████████| 334/334 [1:11:11<00:00, 12.79s/it]
d:\Semester 10\NLP\project\newnlproj\nlp\lib\site-packages\transformers\tokenization_utils_base.py:4072: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
d:\Semester 10\NLP\p

413

In [ ]:
# Save the model
model.save_model("SavedModel")

### Predict

In [ ]:
pred_params = {
        'max_seq_length': 512,
        'use_multiprocessed_decoding': False
        }

model = T5Model('t5', 'outputs/best_model', args=pred_params, use_cuda=cuda)
pred = model.predict(list(test['input_text']))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Generating outputs:   0%|          | 0/105 [00:00<?, ?it/s]d:\Semester 10\NLP\project\newnlproj\nlp\lib\site-packages\transformers\tokenization_utils_base.py:4072: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare

In [ ]:
import os
# Save the model and tokenizer explicitly
model.model.save_pretrained("saved_model")
model.tokenizer.save_pretrained("saved_model")

# Verify saved files
print("Saved files:", os.listdir("saved_model"))

Saved files: ['added_tokens.json', 'config.json', 'generation_config.json', 'model.safetensors', 'special_tokens_map.json', 'spiece.model', 'tokenizer_config.json']


In [ ]:
# Load the model from the 'saved_model' directory
modelLoaded = T5Model('t5', "saved_model", args=pred_params, use_cuda=cuda)
predLoaded = modelLoaded.predict(list(test['input_text']))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Generating outputs:   0%|          | 0/105 [00:00<?, ?it/s]d:\Semester 10\NLP\project\newnlproj\nlp\lib\site-packages\transformers\tokenization_utils_base.py:4072: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare

## Evaluation of the model <a class="anchor" id="Evaluation-of-the-model"></a>

<!-- <img src="https://raw.githubusercontent.com/chakki-works/sumeval/master/doc/top.png" style="height: 150px; width: 600px;  object-position: 0px;"/> -->

Evaluate the model performance with the [sumeval]'s Rouge score.

[sumeval]: https://github.com/chakki-works/sumeval

Rouge1: Evaluate the generated text in units of bi-grams.

Rouge2: Evaluate the generated text in units of uni-grams.

RougeL: Evaluate the match of the generated text sequence.

ORG Text VS ORG summaries

In [ ]:
rouge = RougeCalculator(stopwords=True, lang="en")
target_text_list = test["target_text"].tolist()

def rouge_calc(preds, targets):
    rouge_1 = [rouge.rouge_n(summary=preds[i],references=targets[i],n=1) for i in range(len(preds))]
    rouge_2 = [rouge.rouge_n(summary=preds[i],references=targets[i],n=2) for i in range(len(preds))]
    rouge_l = [rouge.rouge_l(summary=preds[i],references=targets[i]) for i in range(len(preds))]

    return {"Rouge_1": np.array(rouge_1).mean(),
            "Rouge_2": np.array(rouge_2).mean(),
            "Rouge_L": np.array(rouge_l).mean()}
rouge_calc(target_text_list, list(test['input_text']))

{'Rouge_1': 0.1569710241656346,
 'Rouge_2': 0.07369900847764918,
 'Rouge_L': 0.10276240757247822}

ORG Text VS predicted summaries

In [ ]:

rouge_calc(predLoaded, list(test['input_text']))

{'Rouge_1': 0.035533593614212496,
 'Rouge_2': 0.016312304179078196,
 'Rouge_L': 0.03333874052173706}

ORG text VS T5 prediction without training

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import pandas as pd

# Load pre-trained T5 model and tokenizer
model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Assuming test is your DataFrame and "input_text" is the column containing input texts
input_texts = test["input_text"].tolist()

# Function to generate summaries using T5 model
def generate_summary(text):
    input_ids = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(input_ids, num_beams=4, min_length=30, max_length=150, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Generate summaries for input texts
summaries = [generate_summary(test) for test in input_texts]

# Add the generated summaries to the DataFrame
test["summary"] = summaries
t5Predict = test["summary"].tolist()

rouge = RougeCalculator(stopwords=True, lang="en")

rouge_calc(t5Predict, list(test['input_text']))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
